<a href="https://colab.research.google.com/github/Yogeshkumar170201/Yogeshkumar170201/blob/master/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ykdahiya","key":"79b26f11d46af028bf83aa38ff0920af"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d hasyimabdillah/workoutfitness-video

100% 4.32G/4.32G [02:39<00:00, 27.0MB/s]
100% 4.32G/4.32G [02:39<00:00, 29.1MB/s]


In [7]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [8]:
!unzip /content/workoutfitness-video.zip -d /content/datset

Archive:  /content/workoutfitness-video.zip
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_1.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_10.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_11.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_12.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_13.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_14.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_15.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_16.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_17.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_18.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_19.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_2.mp4  
  inflating: /content/

In [9]:
import cv2
import numpy as np

def jitter_video(video_path, file_name, class_name, brightness_range=(0.5, 1.5), contrast_range=(0.5, 1.5), 
                 saturation_range=(0.3, 1.0), hue_range=(-10, 10), resolution=(640, 360), output_fps=30):
    # Load the video
    cap = cv2.VideoCapture(video_path)

    # Define the codec and video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    filename = os.path.basename(file_name)
    filename = os.path.splitext(filename)[0]
    output_file = os.path.join('/content/datset',class_name, filename + '_jittered.mp4')
    writer = cv2.VideoWriter(output_file, fourcc, output_fps, (resolution[0], resolution[1]))

    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            # Resize the frame to the specified resolution
            frame = cv2.resize(frame, resolution)

            # Jittering
            brightness = np.random.uniform(brightness_range[0], brightness_range[1])
            contrast = np.random.uniform(contrast_range[0], contrast_range[1])
            saturation = np.random.uniform(saturation_range[0], saturation_range[1])
            hue = np.random.uniform(hue_range[0], hue_range[1])
            frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            frame_hsv[:, :, 1] = cv2.multiply(frame_hsv[:, :, 1], saturation)
            frame_hsv[:, :, 0] = cv2.add(frame_hsv[:, :, 0], hue)
            frame_hsv[:, :, 2] = cv2.multiply(frame_hsv[:, :, 2], brightness)
            frame_jittered = cv2.cvtColor(frame_hsv, cv2.COLOR_HSV2BGR)
            frame_jittered = cv2.merge((cv2.multiply(frame_jittered[:,:,0], 0.9), 
                                        cv2.multiply(frame_jittered[:,:,1], contrast),
                                        cv2.multiply(frame_jittered[:,:,2], contrast)))

            # Write the frame to the output video
            writer.write(frame_jittered)

        else:
            break

    cap.release()
    writer.release()


In [10]:
import os
# Specify the directory containing the dataset. 
DATASET_DIR = "datset"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = os.listdir('/content/datset')

In [11]:
for class_index, class_name in enumerate(CLASSES_LIST):
  files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
  # Iterate through all the files present in the files list.
  for file_name in files_list:
    # Get the complete video path.
    video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
    jitter_video(video_file_path, file_name, class_name)

In [14]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 35.7 MB/s eta 0:00:00


In [15]:
import tensorflow as tf
import numpy as np
import cv2
import os
import mediapipe as mp
from google.colab.patches import cv2_imshow
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping

In [16]:
mp_holistic = mp.solutions.holistic

In [17]:
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 10

In [18]:
def landmarks_extraction(path):
  # Declare a list to store video frames.
  frames_list = []
  # Read the Video File using the VideoCapture object.
  video_reader = cv2.VideoCapture(path)

  # Get the total number of frames in the video.
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  # Calculate the the interval after which frames will be added to the list.
  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

  # Iterate through the Video Frames.
  
  for frame_counter in range(SEQUENCE_LENGTH):

      # Set the current frame position of the video.
      video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

      # Reading the frame from the video. 
      success, frame = video_reader.read() 

      # Check if Video frame is not successfully read then break the loop
      if not success:
          break
      # Append the normalized frame into the frames list
      frames_list.append(frame)

  # Release the VideoCapture object. 
  video_reader.release()
  marks = []
  with mp_holistic.Holistic() as holistic:
    for i in frames_list:
      # Convert frame to RGB
      frame = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)

      # Detect poses in frame
      results = holistic.process(frame)
      pose_landmarks = results.pose_landmarks
      # print(pose_landmarks)
      if pose_landmarks is not None:
        landmarks = [(lmk.x, lmk.y, lmk.z) for lmk in pose_landmarks.landmark]
        marks.append(np.array(landmarks))
  # Return the frames list.
  marks = np.array(marks)
  return marks

In [19]:
# Specify the directory containing the dataset. 
DATASET_DIR = "datset"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = os.listdir('/content/datset')
NO_OF_CATEGORIES = 5
CLASSES_LIST = CLASSES_LIST[:NO_OF_CATEGORIES]

In [20]:
def create_dataset():
  marks = []
  labels = []
  for class_index, class_name in enumerate(CLASSES_LIST):
    print(class_index, class_name)
    # Display the name of the class whose data is being extracted.
    print(f'Extracting Data of Class: {class_name}')

    # Get the list of video files present in the specific class name directory.
    files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

    # Iterate through all the files present in the files list.
    for file_name in files_list:

      # Get the complete video path.
      video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

      # Extract the landmarks of the video file.
      landmarks = landmarks_extraction(video_file_path)
      marks.append(np.array(landmarks))
      labels.append(class_index)
  return marks, labels

In [21]:
marks, labels = create_dataset()

0 deadlift
Extracting Data of Class: deadlift
1 bench press
Extracting Data of Class: bench press
2 russian twist
Extracting Data of Class: russian twist
3 hip thrust
Extracting Data of Class: hip thrust
4 shoulder press
Extracting Data of Class: shoulder press


In [61]:
labels = np.array(labels)
marks = np.array(marks)
x = marks
y = labels

In [62]:
print(x.shape)

(282,)


In [63]:
for i in range(0, len(x)):
  if x[i].size!=0:
    x[i] = x[i].reshape(x[i].shape[0], -1)

In [64]:
print(x.shape)

(282,)


In [65]:
for i in reversed(range(len(x))):
  if x[i].size==0:
    x = np.delete(x, i)
    y = np.delete(y, i)

In [66]:
for i in reversed(range(len(x))):
  if x[i].shape[0]<SEQUENCE_LENGTH:
    x = np.delete(x, i)
    y = np.delete(y, i)
  elif x[i].shape[0]>=SEQUENCE_LENGTH:
    x[i] = x[i][:SEQUENCE_LENGTH, ]

In [67]:
print(x.shape)
print(y.shape)

(170,)
(170,)


In [69]:
z = y
print(y.shape)

(170,)


In [70]:
z = tf.keras.utils.to_categorical(z).astype(int)
print(z.shape)

(170, 5)


In [71]:
print(z)

[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]

In [225]:
# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32*NO_OF_CATEGORIES, input_shape=(SEQUENCE_LENGTH, 99)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16*NO_OF_CATEGORIES, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(8*NO_OF_CATEGORIES, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4*NO_OF_CATEGORIES, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2*NO_OF_CATEGORIES, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NO_OF_CATEGORIES, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 160)               166400    
                                                                 
 dropout_52 (Dropout)        (None, 160)               0         
                                                                 
 dense_52 (Dense)            (None, 80)                12880     
                                                                 
 dropout_53 (Dropout)        (None, 80)                0         
                                                                 
 dense_53 (Dense)            (None, 40)                3240      
                                                                 
 dropout_54 (Dropout)        (None, 40)                0         
                                                                 
 dense_54 (Dense)            (None, 20)              

In [226]:
X_train, X_test, y_train, y_test = train_test_split(x, z, test_size=0.25, random_state=42)

In [227]:
for i in range(len(X_train)):
  X_train[i] = tf.constant(X_train[i])
for i in range(len(X_test)):
  X_test[i] = tf.constant(X_test[i])

In [228]:
x_train = tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)
x_test = tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)

for i in range(len(X_train)):
  x_train = x_train.write(i, tf.constant(X_train[i]))

for i in range(len(X_test)):
  x_test = x_test.write(i, tf.constant(X_test[i]))
y_train = tf.constant(y_train)
y_test = tf.constant(y_test)
print(type(x_train))
print(type(x_test))

<class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>
<class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>


In [229]:
x_train = x_train.stack()
x_test = x_test.stack()
y_train = y_train.stack()
y_test = y_test.stack()

AttributeError: ignored

In [230]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=500, batch_size=40, callbacks=[early_stopping])

Epoch 1/500
4/4 [==============================] - 8s 452ms/step - loss: 1.6191 - accuracy: 0.1618 - val_loss: 1.6081 - val_accuracy: 0.0588
Epoch 2/500
4/4 [==============================] - 0s 17ms/step - loss: 1.6053 - accuracy: 0.1691 - val_loss: 1.5845 - val_accuracy: 0.1471
Epoch 3/500
4/4 [==============================] - 0s 19ms/step - loss: 1.5668 - accuracy: 0.2868 - val_loss: 1.5426 - val_accuracy: 0.5000
Epoch 4/500
4/4 [==============================] - 0s 16ms/step - loss: 1.5602 - accuracy: 0.3015 - val_loss: 1.5016 - val_accuracy: 0.5588
Epoch 5/500
4/4 [==============================] - 0s 16ms/step - loss: 1.5485 - accuracy: 0.3382 - val_loss: 1.4813 - val_accuracy: 0.4706
Epoch 6/500
4/4 [==============================] - 0s 16ms/step - loss: 1.5244 - accuracy: 0.2500 - val_loss: 1.4462 - val_accuracy: 0.5000
Epoch 7/500
4/4 [==============================] - 0s 16ms/step - loss: 1.4925 - accuracy: 0.3235 - val_loss: 1.4077 - val_accuracy: 0.5294
Epoch 8/500
4/4 [==

In [231]:
loss, accuracy = model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 12ms/step - loss: 0.6625 - accuracy: 0.8235


In [232]:
print(loss)
print(accuracy)

0.6624969244003296
0.8235294222831726


In [233]:
y_pred = model.predict(x_test)

2/2 [==============================] - 1s 7ms/step


In [234]:
print(y_test)

tf.Tensor(
[[0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]], shape=(34, 5), dtype=int64)


In [235]:
print(y_pred.shape)

(34, 5)


In [236]:
y_predd = []
for i in range(0, y_pred.shape[0]):
  label = 0
  val = 2.2250738585072014e-308
  for j in range(0, NO_OF_CATEGORIES):
    if y_pred[i][j] > val:
      val = y_pred[i][j]
      label = j
  y_predd.append(label)

In [237]:
y_true = []
for i in range(0, y_pred.shape[0]):
  for j in range(0, NO_OF_CATEGORIES):
    if y_test[i][j] == 1:
      y_true.append(j)

In [238]:
print(y_predd)

[4, 0, 3, 0, 4, 4, 4, 1, 3, 1, 0, 4, 1, 1, 1, 1, 0, 3, 1, 2, 0, 0, 2, 0, 0, 0, 1, 3, 1, 1, 0, 1, 4, 3]


In [239]:
print(y_true)

[4, 0, 3, 0, 4, 4, 4, 1, 2, 1, 0, 4, 3, 1, 1, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 4, 2]


In [240]:
cm = confusion_matrix(y_true, y_predd)

In [241]:
print(classification_report(y_true, y_predd))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95        11
           1       0.91      1.00      0.95        10
           2       0.50      0.20      0.29         5
           3       0.20      0.50      0.29         2
           4       1.00      1.00      1.00         6

    accuracy                           0.82        34
   macro avg       0.72      0.72      0.70        34
weighted avg       0.85      0.82      0.82        34

